<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/Resume_GPT_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Version of GPT Resume Seach Tool

## Installs

In [ ]:
!python3 -m pip install --upgrade langchain faiss-cpu chromadb openai tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.2/892.2 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.5/249.5 kB 14.3 MB/s eta 0:00:00


## Imports and Google Drive Mount

In [ ]:
# set up google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [ ]:
import argparse, json, time, datetime, openai
from pathlib import Path

In [ ]:
def set_open_ai_key(env_path=None):
  #import json, os
  #from pathlib import Path
  try:
    with open(env_path, "r") as f:
        env_vars = json.load(f)
    os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    #os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
    openai.Model.list() #test a random command on the openai API
    return True
  except Exception as e:
    print(e)
  return False

## Setup

In [ ]:
# setup API key
openai_env_path, openai.api_key = None, None
cwd = Path.cwd()
# resume_path = cwd / "Resumes"
# resume_path.mkdir(exist_ok=True)

openai_env_path = cwd/ "drive/MyDrive/Colab Notebooks/openai.env"
set_open_ai_key(openai_env_path)

True

## Load and Parse Resume Books

In [ ]:
#cwd = Path.cwd()
#output_path = cwd / "drive/MyDrive/Colab Notebooks/Output"

In [ ]:
#Set up document loader for pdf resume books
embeddings = OpenAIEmbeddings()
resume_path1 = "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"
resume_path2 = "/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf"

In [ ]:
def load_resumes(path, skip_pages):
  loader = PyPDFLoader(path)
  pages = loader.load_and_split()
  pages_clean = pages[skip_pages:]
  return pages_clean

In [ ]:
# actual resumes start on page 2 of this pdf compilation
r1 = load_resumes(resume_path1, 2)

In [ ]:
# actual resumes start on page 1 of this pdf compilation
r2 = load_resumes(resume_path2, 1)

In [ ]:
r1[0]

Document(page_content='YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfoliotheory,riskmanagement,financialdata science and machine learning, time series analysis, interest rate modelingUNIVERSITY OF WASHINGTONSeattle, WABS in Mathematics(Sep. 2017 – Jun. 2021)●Coursework:Probability, linear algebra, numericalanalysis, statistics, ODEs and PDEs, measure theory●Honors:Magna Cum Laude (Top 3.5%), Dean’s ListEXPERIENCECHINA CONSTRUCTION BANK, NEW YORK BRANCHNew York, NYQuantitative Risk Analyst(Jun. 2022 – Aug. 2022)●Builtacountryriskpredictorleveraginglinearmodels,boosting,randomforestbasedonS&Pdataofeconomics and political factors, and achieved 87.2% in-sample and 78.9% out-of-sample accuracy●Drafted country risk report for the US collaborative

In [ ]:
r1[0].metadata["name"] = "YIN FU"

In [ ]:
r1[1]

Document(page_content='ARJUN KALSI\narjun.kalsi@nyu.edu\n■\nlinkedin.com/in/arjunkalsi1\n■\ngithub.com/arjunkalsi\n. \nEDUCA TION\nNEW YORK UNIVERSITY\nNew\nYork, NY\nThe Courant Institute of Mathematical Sciences\nMS in Mathematics in Finance\n(expected-Dec 2022) \n●\nCoursework:\nMonte\nCarlo\nmethods,\nBrownian\nmotion,\nsupervised/unsupervised\nlearning,\nfeature map regression, cross-validation, neural networks, data cleaning and web-scraping\nUNIVERSITY OF WARWICK\nCoventry, UK\nBS Mathematics, Operational Research, Statistics, Economics\n(Sep 2018-Jun 2021)\n●\nCoursework:\nmathematical analysis, linear algebra,\nprobability, Bayesian statistics and decision\ntheory, MLE,  options pricing, linear statistical modeling, stochastic processes\nEXPERIENCE\nSOFR ACADEMY -\nQuantitative Analyst Intern\n(Dec 2021-June\n2022)\nNew York, NY\n●\nConstructed a publication handbook for the firm’s new Across-the-Curve Credit Spread Index \n(AXI) tool, explaining all automated code related to 

In [ ]:
r1[0]

Document(page_content='YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfoliotheory,riskmanagement,financialdata science and machine learning, time series analysis, interest rate modelingUNIVERSITY OF WASHINGTONSeattle, WABS in Mathematics(Sep. 2017 – Jun. 2021)●Coursework:Probability, linear algebra, numericalanalysis, statistics, ODEs and PDEs, measure theory●Honors:Magna Cum Laude (Top 3.5%), Dean’s ListEXPERIENCECHINA CONSTRUCTION BANK, NEW YORK BRANCHNew York, NYQuantitative Risk Analyst(Jun. 2022 – Aug. 2022)●Builtacountryriskpredictorleveraginglinearmodels,boosting,randomforestbasedonS&Pdataofeconomics and political factors, and achieved 87.2% in-sample and 78.9% out-of-sample accuracy●Drafted country risk report for the US collaborative

In [ ]:
# combine resume books
resumes = r1+r2

In [ ]:
len(resumes)

118

In [ ]:
# start with one resume book
#loader = PyPDFLoader(resume_path1)
#pages = loader.load_and_split()

In [ ]:
# actual resumes start on page 2 of this pdf compilation
#resumes = pages[2:]

## Chunk Resumes

In [ ]:
# split the documents into chunks
#text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
#texts = text_splitter.split_documents(resumes)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set small chunk size, just to test.
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
)

In [ ]:
texts = text_splitter.split_documents(resumes)
print(texts[0])

page_content='YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfoliotheory,riskmanagement,financialdata science and machine learning, time series analysis, interest rate modelingUNIVERSITY OF WASHINGTONSeattle, WABS in Mathematics(Sep. 2017 – Jun.' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf', 'page': 2}


## Use Vector Stores to create embeddings and preform similarity search

to do: figure out which vector store is best to use - FAISS vs Chroma

In [ ]:
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

### Using FAISS

In [ ]:
faiss_index = FAISS.from_documents(texts, OpenAIEmbeddings())

In [ ]:
docs = faiss_index.similarity_search("knows statistics", k=8)

In [ ]:
for doc in docs:
    #print(str(doc.metadata["page"]) + ":", doc.page_content)
    print(str(doc.metadata["page"]) + ":", doc.page_content[:100] + ":", str(doc.metadata["source"]))

5: Coursework:
probability, stochastic processes, time
series analysis, numerical analysis, statistical: /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf
2: 2021)●Coursework:Probability, linear algebra, numericalanalysis, statistics, ODEs and PDEs, measure : /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf
19: • Gained proficiency with statistical analysis software, SPSS , and statistical testing of scientifi: /content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf
17: previously. In addition, we recommend an intermediate course on mathematical statistics or engineeri: /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf
20: real-world datasets and model them in Python using techniques from statistics, quantitative finance : /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf
25: Relevant Coursework : Statistic s, Information Systems, Microe conomics , Macroeconomics, Calculus  : 

In [ ]:
docs[5]

Document(page_content='Relevant Coursework : Statistic s, Information Systems, Microe conomics , Macroeconomics, Calculus  \n \nEXPERIENCE  \nFordham Gabelli Consulting Challenge                                                                                                                   Bronx, NY  \nTeam Member, Report Coordinator                                                                                                             August  2022  - Present', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf', 'page': 25})

## Call OpenAI to use GPT to answer questions based on Resumes

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
#question = "Who knows statistics?"
question = "which resumes have statistics skills?"

In [ ]:
# test different prompts
context = " Make a table of results from the documents given with columns indicating page and source."
#context = " Make a list of the page numbers from the documents given, using the 'page' metadata, remove all duplicates from the list"
#context = " make a table using the document metadata table with columns: page, source"
#context = " Make a list from the documents given."
#context = " return the document metadata"

In [ ]:
# make output in json with the following source, page, knows statistics 
#context = " Generate a list of resulting resumes with page numbers and source file information, provide them in JSON format with the following keys: page_id, source_file"
#context = " Generate a list of resulting resumes with their metadata fields, provide them in JSON format with the following keys: page_id, source_file"
context = " Provide resulting resumes in JSON format with the following keys from the document metadata fields: page_id, source_file"

In [ ]:
query = question + context
print(query)

which resumes have statistics skills? Provide resulting resumes in JSON format with the following keys from the document metadata fields: page_id, source_file


In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

In [ ]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The resumes with statistics skills are the ones on page 1, page 2, page 3, page 4, and page 5 of the resume book from /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf.\n\nThe JSON format for the list of resumes with statistics skills is:\n[\n  {\n    "page_id": 1,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"\n  },\n  {\n    "page_id": 2,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"\n  },\n  {\n    "page_id": 3,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"\n  },\n  {\n    "page_id": 4,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/'}

In [ ]:
#best one?
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The resumes with statistics skills are from the following sources: \n/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf\n/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf\n\nThe resumes with their metadata fields in JSON format are as follows: \n[{"page_id": 0, "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"}, {"page_id": 1, "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"}, {"page_id": 2, "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf"}, {"page_id": 3, "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"}, {"page_id": 4, "source_'}

### Using Chroma

In [ ]:
# chain = load_qa_with_sources_chain(llm=OpenAI(), chain_type="stuff")
# chain({"input_documents": docs, "question": query}, return_source_documents=True)

In [ ]:
# db = DeepLake(dataset_path=dataset_path, read_only=True, embedding_function=embeddings)

# retriever = db.as_retriever()
# retriever.search_kwargs['distance_metric'] = 'cos'
# retriever.search_kwargs['k'] = 4

# qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)

# # What was the restaurant the group was talking about called?
# query = input("Enter query:")

# # The Hungry Lobster
# ans = qa({"query": query})

# print(ans)

In [ ]:
# def qa(file, query, chain_type, k):
#     # load document
#     loader = PyPDFLoader(file)
#     documents = loader.load()
#     # split the documents into chunks
#     text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#     texts = text_splitter.split_documents(documents)
#     # select which embeddings we want to use
#     embeddings = OpenAIEmbeddings()
#     # create the vectorestore to use as the index
#     db = Chroma.from_documents(texts, embeddings)
#     # expose this index in a retriever interface
#     retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
#     # create a chain to answer questions 
#     qa = RetrievalQA.from_chain_type(
#         llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
#     result = qa({"query": query})
#     print(result['result'])
#     return result

In [ ]:
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 8})

In [ ]:
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=retriever, return_source_documents=False)
result = qa({"query": query})
print(result['result'])

 The resumes with statistics skills are: 

Data Analysis Intern (page 1, source file 1), 
Data Science Leadership and Activities (page 5, source file 1), 
Covid19 Data Visualization in Python (page 7, source file 1), 
Huijin Asset Management (page 9, source file 1).

The JSON format would look like this: 
[
  {
    "page_id": 1,
    "source_file": 1
  },
  {
    "page_id": 5,
    "source_file": 1
  },
  {
    "page_id": 7,
    "source_file": 1
  },
  {
    "page_id": 9,
    "source_file": 1
  }
]


In [ ]:
query

'which resumes have statistics skills? Provide resulting resumes in JSON format with the following keys from the document metadata fields: page_id, source_file'

In [ ]:
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=retriever, return_source_documents=False)
result = qa({"query": query})
print(result['result'])

 The resumes with statistics skills are the resumes with the page_ids of 1, 4, and 5. The JSON format for the resulting resumes is: 
[
  {
    "page_id": 1,
    "source_file": "resume1.pdf"
  },
  {
    "page_id": 4,
    "source_file": "resume2.pdf"
  },
  {
    "page_id": 5,
    "source_file": "resume3.pdf"
  }
]


In [ ]:
# try prompt templates?